In [2]:
import PySimpleGUI as sg

####### THEME #######
sg.theme('DarkGray1')
#####################

#LAYOUT
layout = [
    [sg.Radio('Target', 'tgt' , key='-TARGET_RADIO-', default=True), sg.Input('', key='-TARGET_INPUT-', size=(45, 0), pad=(51, 5))],
    [sg.Radio('List of Targets', 'tgt', key='-TARGET_LIST_RADIO-'), sg.Input(key='-TARGET_FILE-'), sg.FileBrowse(enable_events=True, size=(12, 0))],
    [sg.HSeparator()],
    [sg.Radio('Sector', 'sct', key='-SEC_RADIO-', default=True), sg.Input('', key='-SEC_INPUT-', size=(10, 0), pad=(51, 5))],
    [sg.Radio('List of Sectors', 'sct', key='-SEC_LIST_RADIO-'), sg.Input('', key='-SEC_LIST_INPUT-', pad=(5, 5))],
    [sg.Radio('All Sectors (merged)', 'sct', key='-ALL_SEC_RADIO-')],
    [sg.HSeparator()],
    [sg.Push(), sg.Button('DOWNLOAD', key='-DOWNLOAD_BUTTON-', size=(12, 2)), sg.Push()],
    [sg.ProgressBar(100, size=(500, 15), bar_color=('green', 'lightgrey'))]
    
]


# JANELA
window = sg.Window('TL Download Project', layout=layout, size=(550, 300))


while True:
    evento, valor = window.read()
    
    if evento == sg.WIN_CLOSED or evento == 'Cancelar':
        break
        
window.close()